# Matrix Generation

In [1]:
import sys
sys.path.append("../")
from Recipe import Recipe, Ingredient, RecipeGraph

import settings
import db.db_settings as db_settings
from db.database_connection import DatabaseConnection

import random

import itertools

from tqdm.autonotebook import tqdm

import traceback

import numpy as np

/var/folders/h_/wy6y3sfs1254rd40kbxmvsw00000gn/T/ipykernel_58449/602197836.py:13: TqdmExperimentalWarning:

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)



In [2]:
DatabaseConnection(db_settings.db_host,
                   db_settings.db_port,
                   db_settings.db_user,
                   db_settings.db_pw,
                   db_settings.db_db,
                   db_settings.db_charset)

could not connect to database 'recipe_db' at root@172.17.0.2:3306
Check the configuration in settings.py!


Exception: could not connect to database

In [3]:
%time ids = DatabaseConnection.global_single_query("select id from recipes")

CPU times: user 8.52 s, sys: 719 ms, total: 9.24 s
Wall time: 9.4 s


In [4]:
import AdjacencyMatrix

* create Adjacency Matrix

In [5]:
def add_entries_from_rec_state(rec_state, m_act, m_mix, m_base_act, m_base_mix):
    mix_m, mix_label = rec_state.get_mixing_matrix()
    act_m, act_a, act_i = rec_state.get_action_matrix()

    # create list of tuples: [action, ingredient]
    seen_actions = np.array(list(itertools.product(act_a,act_i))).reshape((len(act_a), len(act_i), 2))

    # create list of tuples [ingredient, ingredient]
    seen_mixes = np.array(list(itertools.product(mix_label,mix_label))).reshape((len(mix_label), len(mix_label), 2))

    seen_actions = seen_actions[act_m == 1]
    seen_mixes = seen_mixes[mix_m == 1]

    seen_actions = set([tuple(x) for x in seen_actions.tolist()])
    seen_mixes = set([tuple(x) for x in seen_mixes.tolist()])
    
    seen_base_actions = set()
    seen_base_mixes = set()
    
    for act, ing in seen_actions:
        m_act.add_entry(act, ing.to_json(), 1)
        if (act, ing._base_ingredient) not in seen_base_actions:
            seen_base_actions.add((act, ing._base_ingredient))
            m_base_act.add_entry(act, ing._base_ingredient, 1)
    
    for x,y in seen_mixes:
        xj = x.to_json()
        yj = y.to_json()
        if xj < yj:
            m_mix.add_entry(xj,yj,1)
            if (x._base_ingredient, y._base_ingredient) not in seen_base_mixes:
                seen_base_mixes.add((x._base_ingredient, y._base_ingredient))
                m_base_mix.add_entry(x._base_ingredient, y._base_ingredient, 1)


In [6]:
m_act = AdjacencyMatrix.adj_matrix()
m_mix = AdjacencyMatrix.adj_matrix(True)
m_base_act = AdjacencyMatrix.adj_matrix()
m_base_mix = AdjacencyMatrix.adj_matrix(True)

In [7]:
%%time
for i in tqdm(range(100000)):
    try:
        id = random.choice(ids)['id']
        rec = Recipe(id)
        #rec.display_recipe()
        ing = rec.extract_ingredients()
        if len(ing) == 0:
            print(f"warning: recipe {id} has no ingredient! skipping it")
            continue
        rec.apply_instructions(debug=False)
        add_entries_from_rec_state(rec._recipe_state, m_act, m_mix, m_base_act, m_base_mix)
    except Exception as e:
        print("an error occured : " + str(e))
        traceback.print_exc()

an error occured : 'NoneType' object has no attribute 'lower'


Traceback (most recent call last):
  File "<timed exec>", line 4, in <module>
  File "/home/jonas/Dokumente/gitRepos/master_thesis/RecipeAnalysis/Recipe.py", line 888, in __init__
    self.annotate_sentences()
  File "/home/jonas/Dokumente/gitRepos/master_thesis/RecipeAnalysis/Recipe.py", line 981, in annotate_sentences
    self._annotate_sentences(self._sentences, self.predict_labels())
  File "/home/jonas/Dokumente/gitRepos/master_thesis/RecipeAnalysis/Recipe.py", line 933, in predict_labels
    features = [sent2features(sent) for sent in self._sentences]
  File "/home/jonas/Dokumente/gitRepos/master_thesis/RecipeAnalysis/Recipe.py", line 933, in <listcomp>
    features = [sent2features(sent) for sent in self._sentences]
  File "../Tagging/crf_data_generator.py", line 87, in sent2features
    return [word2features(sent, i) for i in range(len(sent))]
  File "../Tagging/crf_data_generator.py", line 87, in <listcomp>
    return [word2features(sent, i) for i in range(len(sent))]
  File "

an error occured : 'NoneType' object is not subscriptable


Traceback (most recent call last):
  File "<timed exec>", line 4, in <module>
  File "/home/jonas/Dokumente/gitRepos/master_thesis/RecipeAnalysis/Recipe.py", line 888, in __init__
    self.annotate_sentences()
  File "/home/jonas/Dokumente/gitRepos/master_thesis/RecipeAnalysis/Recipe.py", line 981, in annotate_sentences
    self._annotate_sentences(self._sentences, self.predict_labels())
  File "/home/jonas/Dokumente/gitRepos/master_thesis/RecipeAnalysis/Recipe.py", line 933, in predict_labels
    features = [sent2features(sent) for sent in self._sentences]
  File "/home/jonas/Dokumente/gitRepos/master_thesis/RecipeAnalysis/Recipe.py", line 933, in <listcomp>
    features = [sent2features(sent) for sent in self._sentences]
  File "../Tagging/crf_data_generator.py", line 87, in sent2features
    return [word2features(sent, i) for i in range(len(sent))]
  File "../Tagging/crf_data_generator.py", line 87, in <listcomp>
    return [word2features(sent, i) for i in range(len(sent))]
  File "

an error occured : 'NoneType' object has no attribute 'lower'


Traceback (most recent call last):
  File "<timed exec>", line 4, in <module>
  File "/home/jonas/Dokumente/gitRepos/master_thesis/RecipeAnalysis/Recipe.py", line 888, in __init__
    self.annotate_sentences()
  File "/home/jonas/Dokumente/gitRepos/master_thesis/RecipeAnalysis/Recipe.py", line 981, in annotate_sentences
    self._annotate_sentences(self._sentences, self.predict_labels())
  File "/home/jonas/Dokumente/gitRepos/master_thesis/RecipeAnalysis/Recipe.py", line 933, in predict_labels
    features = [sent2features(sent) for sent in self._sentences]
  File "/home/jonas/Dokumente/gitRepos/master_thesis/RecipeAnalysis/Recipe.py", line 933, in <listcomp>
    features = [sent2features(sent) for sent in self._sentences]
  File "../Tagging/crf_data_generator.py", line 87, in sent2features
    return [word2features(sent, i) for i in range(len(sent))]
  File "../Tagging/crf_data_generator.py", line 87, in <listcomp>
    return [word2features(sent, i) for i in range(len(sent))]
  File "


CPU times: user 2h 2min 39s, sys: 43.4 s, total: 2h 3min 23s
Wall time: 2h 4min 37s


In [10]:
i

99999

In [11]:
import dill

In [12]:
dill.dump(m_act, file=open("m_act_raw.dill", 'wb'))
dill.dump(m_mix, file=open("m_mix_raw.dill", 'wb'))
dill.dump(m_base_act, file=open("m_base_act_raw.dill", 'wb'))
dill.dump(m_base_mix, file=open("m_base_mix_raw.dill", 'wb'))

In [13]:
m_act.apply_threshold(10)
m_mix.apply_threshold(10)
m_base_act.apply_threshold(20)
m_base_mix.apply_threshold(20)

In [14]:
c_mix = m_mix.get_csr()
c_act = m_act.get_csr()
c_base_mix = m_base_mix.get_csr()
c_base_act = m_base_act.get_csr()

In [15]:
m_mix.compile()
m_act.compile()
m_base_mix.compile()
m_base_act.compile()


In [16]:
dill.dump(m_act, file=open("m_act.dill", 'wb'))
dill.dump(m_mix, file=open("m_mix.dill", 'wb'))
dill.dump(m_base_act, file=open("m_base_act.dill", 'wb'))
dill.dump(m_base_mix, file=open("m_base_mix.dill", 'wb'))

In [17]:
print(c_act.shape, c_mix.shape)
print(len(c_act.nonzero()[0]),len(c_mix.nonzero()[0]))
print(c_base_act.shape, c_base_mix.shape)
print(len(c_base_act.nonzero()[0]),len(c_base_mix.nonzero()[0]))

(64, 237792) (266772, 266772)
13170 17041
(64, 11455) (13528, 13528)
6083 10775


In [18]:
print(c_act.shape, c_mix.shape)
print(len(c_act.nonzero()[0]),len(c_mix.nonzero()[0]))

(64, 237792) (266772, 266772)
13170 17041


In [19]:
np.sum(c_act.toarray() > 1)

13170

* values after 100:
```
(53, 1498) (1620, 1620)
1982 6489
```

* after 1000:
```
(60, 9855) (10946, 10946)
15446 59943
```

* after 10000:
```
(65, 65235) (72448, 72448)
114808 546217
```